- The lightcurve


In [ ]:
import stingray
from stingray import EventList
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
%matplotlib inline

In [ ]:
filename  = ".."                                # opens input fits file
evt       = EventList.read(filename) #, "hea")  # reads the event list. Parameter "hea" refers to "heasarc-supported missions" 

In [ ]:
## Choose energy bands. For nicer data we can directly give energy as an input (Stingray makes the conversion), otherwise use PI channel

eband_soft = [.., ..]   
eband_hard = [.., ..]
eband_tot  = [.., ..]

In [ ]:
## Filter events in selected energy bands

evt_soft = evt.filter_energy_range(eband_soft)  
evt_hard = evt.filter_energy_range(eband_hard)
evt_tot  = evt.filter_energy_range(eband_tot)


--> Extract a light curve using different time bin sizes to show that larger time bins dilute fast variations in the observed signal.

In [ ]:
dt1  = ..                     # choose time bin (in seconds)
lc1 = evt_tot.to_lc(dt = dt1) # extracts lightcurve (attributes: lc.counts or lc.countrate)

In [ ]:
dt2  = ..                     # choose time bin (in seconds)
lc2 = evt_tot.to_lc(dt = dt2) # extracts lightcurve (attributes: lc.counts or lc.countrate)

In [ ]:
## Plot lightcurves

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)  
ax1.plot(lc1.time,lc1.countrate, label=rf"$\Delta$t = {lc1.dt} s")
ax2.plot(lc2.time,lc2.countrate, label=rf"$\Delta$t = {lc2.dt} s")
ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Cts/s")
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Cts/s")
ax1.legend()
ax2.legend()
plt.show()
plt.close("all")

- The power spectrum (rebinning)

In [ ]:
from stingray import AveragedPowerspectrum

--> Show that averaging periodograms reduces the scatter. 

1) To show that a single periodogram is highly scattered around the underlying power spectrum, we give as an input a segment size as large as the length of the largest GTI interval (this selection returns only one segment):

In [ ]:
gti_length = evt_tot.gti[:,1]-evt_tot.gti[:,0]  # compute the length of GTI segments
Tseg       = np.max(gti_length)                 # select the longest
dt         = ..                                 # choose time bin 
norm       = '..'                               # choose a power spectrum normalization 

psd_unbin  = AveragedPowerspectrum.from_events(evt_tot, segment_size=Tseg, dt = dt, norm = norm, gti=evt_tot.gti) 

In [ ]:
frac_err = psd_unbin.power_err/psd_unbin.power          # compute the fractional error
print("Average fractional error: ", np.mean(frac_err))  # print the average fractional error

2. Now we average over more segments (simply by reducing the segment size):

In [ ]:
Tseg     = 50                                           # choose a small segment size 
psd      = AveragedPowerspectrum.from_events(evt_tot, segment_size=Tseg, dt = dt, norm = norm, gti=evt_tot.gti)  

In [ ]:
frac_err = psd.power_err/psd.power                      # compute the fractional error
print("Number of segments:", psd.m)                     # print number of averaged segments 
print("Average fractional error: ", np.mean(frac_err))  # print average fractional error

Alternatively, or in addition, we can reduce the scatter by rebinning a periodogram into larger frequency bins (in doing so we are still averaging periodograms!). 

--> Consider the unbinned periodogram, rebin it logarithmically ($\nu_{i+1}=\nu_i*(1+f)\rightarrow \log(\nu_{i+1})=\log{\nu_i}+\Delta$), and show that the error decreases as the number of averaged frequencies per bin increases.

In [ ]:
psd_reb = psd_unbin.rebin_log(f=...)  # logarithmic rebinning (choose rebinning factor f)

In [ ]:
print("Number of averaged frequencies in each bin: ")
print(psd_reb.m)

In [ ]:
## Compute fractional error and plot it as a function of number of averaged frequencies per bin

frac_err = psd_reb.power_err/psd_reb.power         

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(psd_reb.m, frac_err, color='blue')
ax.set_ylabel("Frac PSD error")
ax.set_xlabel("Number of averaged frequencies")
ax.set_xscale("log")
ax.set_yscale("log")
plt.show()
plt.close("all")

In [ ]:
## Plot rebinned power spectra on top of unrebinned ones

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)                 
ax1.errorbar(psd_unbin.freq,psd_unbin.power,yerr=psd_unbin.power_err,ds='steps-mid', alpha=0.4) 
ax1.errorbar(psd_reb.freq,psd_reb.power,yerr=psd_reb.power_err,ds='steps-mid', color='black')
ax2.errorbar(psd_unbin.freq,psd_unbin.power,yerr=psd_unbin.power_err,ds='steps-mid', alpha=0.4) 
ax2.errorbar(psd.freq,psd.power,yerr=psd.power_err,ds='steps-mid', color='black')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel("Freq (Hz)")
ax1.set_ylabel("Pow")
ax1.set_title("Rebinned in frequency")

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel("Freq (Hz)")
ax2.set_ylabel("Pow")
ax2.set_title("Averaged over segments")

ax1.set_ylim(1, 1e5)
ax2.set_ylim(1,1e5)
plt.show()
plt.close("all")

- The power spectrum (Poisson noise)

In [ ]:
dt           = ..  # choose a small time bin to better sample Poisson noise 
Tseg         = ..   
psd_new      = AveragedPowerspectrum.from_events(evt_tot, segment_size=Tseg, dt = dt, norm = norm, gti=evt_tot.gti)  
psd_reb      = psd_new.rebin_log(f=0.05)


fig, ax = plt.subplots()
ax.errorbar(psd_reb.freq,psd_reb.power,yerr=psd_reb.power_err,ds='steps-mid', alpha=0.4)
ax.axhline(2, linestyle="--")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Pow")
#ax.set_xlim(30, 1/(2*dt))
#ax.set_ylim(1.8,3)
plt.show()
plt.close("all")

In [ ]:
## Select high frequencies where Poisson noise dominates

mask     = psd_reb.freq >= 200          
freq_wn  = psd_reb.freq[mask]
power_wn = psd_reb.power[mask]
err_wn   = psd_reb.power_err[mask]

In [ ]:
## Fit high frequencies with a constant model 

from scipy.optimize import curve_fit   

def constant(x, c):
    return c

cfit, cov = curve_fit(
    constant,
    freq_wn,
    power_wn,
    sigma=err_wn,
)

const_level = cfit[0]
const_err   = np.sqrt(np.diag(cov))    # takes square root of diagonal elements in covariance array (i.e. variances of fitted parameters)

print("Best-fit white noise power:",const_level)
print("Error on white noise power:",const_err[0])

In [ ]:
## Plot power spectrum with fitted Poisson noise level

fig, ax = plt.subplots()               
ax.errorbar(psd_reb.freq,psd_reb.power,yerr=psd_reb.power_err,ds='steps-mid') 
ax.axhline(const_level, linestyle="--")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Pow")
#ax.set_xlim(30, 1/(2*dt))
#ax.set_ylim(1.8,3)
plt.show()
plt.close("all")

- The power spectrum (fractional rms)

In [ ]:
f_min        = ..             # choose max and min frequency
f_max        = ..
rms, err_rms = psd.compute_rms(f_min, f_max, poisson_noise_level=const_level) # does not matter if input power spectrum and poisson noise level are computed in a different normalization
print(rms, err_rms)

- Energy-dependent power spectra

--> Extract power spectra in two different energy bands and determine which one is more variable and on which timescales

In [ ]:
norm     = '..'     

Tseg     = ..          # better choosing a sufficiently long segment size, in order to have access to the lowest frequencies
dt       = ..          # better choosing a high dt to have access to the highest frequency variability to properly fit the Poisson noise power
psd_soft = AveragedPowerspectrum.from_events(evt_soft, segment_size=Tseg, dt = dt, norm = norm, gti=evt_soft.gti)
psd_hard = AveragedPowerspectrum.from_events(evt_hard, segment_size=Tseg, dt = dt, norm = norm, gti=evt_hard.gti)   

psd_reb_soft = psd_soft.rebin_log(f=..)
psd_reb_hard = psd_hard.rebin_log(f=..)

In [ ]:
## Overplot power spectra in the two energy bands 

fig, ax = plt.subplots()   
ax.errorbar(psd_reb_soft.freq,psd_reb_soft.power,yerr=psd_reb_soft.power_err,ds='steps-mid', alpha=0.4, label=f"Soft")
ax.errorbar(psd_reb_hard.freq,psd_reb_hard.power,yerr=psd_reb_hard.power_err,ds='steps-mid', alpha=0.4, label=f"Hard") 
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Pow")
ax.legend()
plt.show()
plt.close("all")

In [ ]:
## Fit Poisson noise

mask     = psd_reb_soft.freq >= 200         
freq_wn  = psd_reb_soft.freq[mask]
power_wn = psd_reb_soft.power[mask]
err_wn   = psd_reb_soft.power_err[mask]

cfit, cov = curve_fit(
    constant,
    freq_wn,
    power_wn,
    sigma=err_wn,
)

const_level_soft = cfit[0]
const_err_soft   = np.sqrt(np.diag(cov)) 

mask     = psd_reb_hard.freq >= 200
freq_wn  = psd_reb_hard.freq[mask]
power_wn = psd_reb_hard.power[mask]
err_wn   = psd_reb_hard.power_err[mask]

cfit, cov = curve_fit(
    constant,
    freq_wn,
    power_wn,
    sigma=err_wn,
)

const_level_hard = cfit[0]
const_err_hard   = np.sqrt(np.diag(cov)) 

print("Best-fit white noise power [soft, hard]:",const_level_soft, const_level_hard)
print("Error on white noise power [soft, hard]:",const_err_soft[0], const_err_hard[0])

In [ ]:
## Plot Poisson noise subtracted power spectra

sigma_soft = np.sqrt(psd_reb_soft.power_err**2+const_err_soft[0]**2)    
sigma_hard = np.sqrt(psd_reb_hard.power_err**2+const_err_hard[0]**2)


fig, ax = plt.subplots()
ax.errorbar(psd_reb_soft.freq,psd_reb_soft.power-const_level_soft,yerr=psd_reb_soft.power_err,ds='steps-mid', color='blue', alpha=0.4, label=f"Soft")
ax.errorbar(psd_reb_hard.freq,psd_reb_hard.power-const_level_hard,yerr=psd_reb_hard.power_err,ds='steps-mid', color='orange', alpha=0.4, label=f"Hard") 
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Pow (rms/mean)^2)")
ax.set_ylim(1e-7,3)
ax.legend()
plt.show()
plt.close("all")

In [ ]:
## Compute fractional rms

f_min        = ..
f_max        = ..
rms_soft, err_rms_soft = psd_reb_soft.compute_rms(f_min, f_max, poisson_noise_level=const_level_soft)
rms_hard, err_rms_hard = psd_reb_hard.compute_rms(f_min, f_max, poisson_noise_level=const_level_hard)
print("Soft band fractional rms (0.01-50 Hz): ",rms_soft, err_rms_soft)
print("Hard band fractional rms (0.01-50 Hz): ",rms_hard, err_rms_hard)

- Cross spectrum

In [ ]:
from stingray import AveragedCrossspectrum

In [ ]:
Tseg  = ..        # choose segment size
dt    = ..        # choose time bin
cs    = AveragedCrossspectrum.from_events(evt_hard, evt_soft, segment_size= Tseg, dt = dt, norm = norm)

In [ ]:
cs_reb = cs.rebin_log(..)


--> Explore different ways of representing the cross spectrum

In [ ]:
## Plot real and imaginary parts, and the squared modulus of the average cross spectrum

fig, (ax1, ax3) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)
ax1.errorbar(cs_reb.freq,cs_reb.power.real,yerr=cs_reb.power_err.real,ds='steps-mid', color="blue")
ax2 = ax1.twinx()
ax2.errorbar(cs_reb.freq,cs_reb.power.imag,yerr=cs_reb.power_err.imag,ds='steps-mid', color="orange")
ax3.errorbar(cs_reb.freq,cs_reb.power,yerr=cs_reb.power_err,ds='steps-mid')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax3.set_xscale('log')
ax3.set_yscale('log')

ax1.set_xlabel("Freq (Hz)")
ax3.set_xlabel("Freq (Hz)")
ax1.set_ylabel("Cross Spectrum", color="blue")
ax2.set_ylabel("Imaginary Cross Spectrum", color="orange")
ax3.set_ylabel("Cross Spectrum Power")
plt.show()
plt.close("all")

In [ ]:
## Plot cross spectrum as a vector in the complex plane

fmin, fmax = .., ..       # Choose frequency range

# Mask the frequency range
mask = (cs_reb.freq >= fmin) & (cs_reb.freq <= fmax)

# Select only the masked points
x = cs_reb.power.real[mask]
y = cs_reb.power.imag[mask]


fig, ax = plt.subplots(figsize=(6,6))

ax.quiver(
    [0]*len(x), [0]*len(y),  # origin for each vector
    x, y,                    # vector components
    angles='xy', scale_units='xy', scale=1,
    color='blue', alpha=0.7
)


xpad = 1 
ypad = 3
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

dx = xmax - xmin
dy = ymax - ymin

ax.set_xlim(xmin - xpad * dx, xmax + xpad * dx)
ax.set_ylim(ymin - ypad * dy, ymax + ypad * dy)

ax.set_xlabel("Real(CS)")
ax.set_ylabel("Imag(CS)")
ax.set_title("Cross-spectrum vectors in the complex plane")
ax.grid(True)
ax.set_aspect('equal') 
plt.show()

- Phase and time lags

In [ ]:
phase, phase_err = cs_reb.phase_lag()
lag,   lag_err   = cs_reb.time_lag()

In [ ]:
## Plot phase and time lags as a function of frequency

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)
ax1.errorbar(cs_reb.freq,phase,yerr=phase_err,ds='steps-mid')
ax2.errorbar(cs_reb.freq,lag,yerr=lag_err,ds='steps-mid')

ax1.set_xscale('log')
ax2.set_xscale('log')

ax1.axhline(0, linestyle="--")
ax2.axhline(0, linestyle="--")

ax1.set_xlabel("Freq (Hz)")
ax1.set_ylabel("Phase (rad)")

ax2.set_xlabel("Freq (Hz)")
ax2.set_ylabel("Lag (s)")

ax1.set_ylim(-0.5,0.5)
ax2.set_ylim(-0.01,0.01)
#ax2.set_yscale('log')

plt.show()
plt.close("all")

--> Determine maximum time/phase lags from phase wrapping and overplot the curves that describe these limits:

In [ ]:
## Compute phase wrapping limits

up_phase  = np.pi * np.ones(len(cs_reb.freq))
low_phase = -np.pi * np.ones(len(cs_reb.freq))

up_lag    = 1./(2*cs_reb.freq)
low_lag   = -1./(2*cs_reb.freq)

In [ ]:
## Plot phase wrapping curves

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)
ax1.errorbar(cs_reb.freq,phase,yerr=phase_err,ds='steps-mid')
ax2.errorbar(cs_reb.freq,lag,yerr=lag_err,ds='steps-mid')

ax1.plot(cs_reb.freq,up_phase, linestyle='--', color='r')
ax1.plot(cs_reb.freq,low_phase, linestyle='--', color='r')

ax2.plot(cs_reb.freq,up_lag, linestyle='--', color='r')
ax2.plot(cs_reb.freq,low_lag, linestyle='--', color='r')

ax1.set_xscale('log')
ax2.set_xscale('log')

ax1.axhline(0, linestyle="--")
ax2.axhline(0, linestyle="--")

ax1.set_xlabel("Freq (Hz)")
ax1.set_ylabel("Phase (rad)")

ax2.set_xlabel("Freq (Hz)")
ax2.set_ylabel("Lag (s)")

#ax1.set_ylim(-2*np.pi,2*np.pi)
#ax2.set_ylim(-0.01,0.01)

plt.show()
plt.close("all")

--> Estimate intrinsic amplitude of soft lag

In [ ]:
cs_rebreb              = cs.rebin_log(0.1)     # we first rebin a bit more in order to reduce scattering in the data
lagreb,   lagreb_err   = cs_rebreb.time_lag()


## Choose a range of frequencies where to search for zero-crossing
fmin = 3     # lower bound in Hz                  
fmax = 100.0 # upper bound in Hz


mask = (cs_rebreb.freq >= fmin) & (cs_rebreb.freq <= fmax)     # Select lag and frequencies in the defined range
freq_sel = cs_rebreb.freq[mask]
lag_sel = lagreb[mask]

In [ ]:
## Interpolate the lag frequency spectrum to obtain a smoother curve for more accurate identification of zero-crossing
## TO BE SUBSTITUTED WITH PHYSICAL MODEL FITTING

from scipy.interpolate import CubicSpline                       

spline    = CubicSpline(freq_sel, lag_sel)                      # cubic spline interpolation
freq_fine = np.linspace(fmin, fmax, 5000)                       # define a fine frequency grid for interpolated lag
lag_fine  = spline(freq_fine)                                   # computes interpolated lag on the frequency grid

sign_changes    = np.where(np.diff(np.sign(lag_fine)))[0]       # detects where the lag changes sign
first_cross_idx = sign_changes[0]                               # selects the first crossing

f1, f2          = freq_fine[first_cross_idx:first_cross_idx+2]  # linearly interpolates between the two frequencies of the first crossing
l1, l2          = lag_fine[first_cross_idx:first_cross_idx+2]
zero_cross_freq = f1 - l1 * (f2 - f1) / (l2 - l1)    

In [ ]:
## Plot lag-frequency spectrum, interpolation curve, and zero-crossing frequency

print("First zero-crossing frequency:", zero_cross_freq, "Hz")

fig, ax1 = plt.subplots(ncols=1, figsize=(8, 4), constrained_layout=True)
ax1.errorbar(cs_rebreb.freq,lagreb,yerr=lagreb_err,ds='steps-mid')
ax1.plot(freq_fine, lag_fine, '-')
ax1.scatter([zero_cross_freq], [0], color='green', s=50, label='Zero crossing')

ax1.axhline(0, linestyle="--")
ax1.set_xscale('log')


ax1.set_xlim(fmin,fmax)
ax1.set_ylim(-0.01,0.01)
ax1.set_xlabel('Frequency')
ax1.set_ylabel('Lag')
ax1.legend()
plt.show()
plt.close("all")

In [ ]:
## Convert zero-crossing frequency to lag amplitude

fzero_min = zero_cross_freq
fzero_max = 60.
tau_intr_max = 1/(2.*fzero_min)
tau_intr_min = 1/(2.*fzero_max)

print(f"Estimated range of intrinsic lag amplitude: {tau_intr_max}-{tau_intr_min} s")


- Coherence (raw)

In [ ]:
coh, coh_err = cs_reb.coherence()  # This computes the "raw" coherence

In [ ]:
## Plot coherence as a function of frequency

fig, ax = plt.subplots()
ax.errorbar(cs_reb.freq,coh,yerr=coh_err,ds='steps-mid')
ax.set_xscale('log')
ax.axhline(0, linestyle="--")
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Raw Coherence")
plt.show()
plt.close("all")

--> Show that the bias term decreases as the number of averaged cross spectra increases

In [ ]:
## Compute bias term

psd_soft_corr = psd_reb_soft.power-const_level_soft
psd_hard_corr = psd_reb_hard.power-const_level_hard

n2 = psd_soft_corr*psd_reb_hard.power + psd_hard_corr*psd_reb_soft.power + const_level_soft*const_level_hard

In [ ]:
## Plot bias terms vs frequency/number of averaged cross spectra

fig, ax1 = plt.subplots()

ax1.scatter(cs_reb.m, n2, color='blue')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(f"Number of averaged cross spectra", color = 'blue')
ax1.set_ylabel(rf"$n^2$")
ax1.tick_params(axis='x', colors='blue')

ax2 = ax1.twiny()
ax2.scatter(cs_reb.freq, n2, color='orange', label='freq')
ax2.set_xscale('log')
ax2.set_xlabel('Freq (Hz)', color = 'orange')
ax2.tick_params(axis='x', colors='orange')


plt.show()
plt.close("all")

- Coherence (intrinsic)

In [ ]:
coh_int  = coh*( psd_reb_soft.power * psd_reb_hard.power)/( psd_soft_corr * psd_hard_corr)

In [ ]:
## Plot intrinsic coherence

fig, ax = plt.subplots()
ax.errorbar(cs_reb.freq,coh_int,yerr=coh_err*0,ds='steps-mid')
ax.set_xscale('log')
ax.set_xlabel("Freq (Hz)")
ax.set_ylabel("Intrinsic Coherence")
ax.set_ylim(0,1)
plt.show()
plt.close("all")

- Lag-energy spectrum

In [ ]:
from stingray import LagSpectrum

In [ ]:
## Choose reference band and channels

ref_band       =[0.5,10]
chan_band      = np.geomspace(0.5, 10, 20)

In [ ]:
freq       = [.., ..]       # choose frequency range for extraction of lag energy spectrum
lagspec    = LagSpectrum(evt, freq_interval=freq, segment_size=Tseg, bin_time=dt, energy_spec=chan_band, ref_band=ref_band)

In [ ]:
## Plot lag-energy spectrum

fig, ax = plt.subplots(1, 1, figsize=(10,8))
ax.errorbar(lagspec.energy, lagspec.spectrum, yerr=lagspec.spectrum_error, label=rf"$\Delta \nu=${freq} Hz")

ax.set_xscale('log')
ax.set_xlim(0.5, 10.)

ax.set_ylabel("Time lag (s)")
ax.set_xlabel("Energy (keV)")
ax.legend()
plt.show()
plt.close("all")

- Covariance and rms spectrum

In [ ]:
from stingray import CovarianceSpectrum, RmsSpectrum

In [ ]:
norm    = ".."  # choose normalization
covspec = CovarianceSpectrum(evt, freq_interval=freq, segment_size=Tseg, bin_time=dt, energy_spec=chan_band, ref_band=ref_band, norm=norm)

In [ ]:
rmsspec = RmsSpectrum(evt, freq_interval=freq, segment_size=Tseg, bin_time=dt, energy_spec=chan_band, ref_band=ref_band, norm=norm)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))
ax.errorbar(covspec.energy, covspec_low.spectrum, yerr=covspec_low.spectrum_error, label=rf"Cov $\Delta \nu=${lowfreq} Hz")
ax.errorbar(rmsspec.energy, rmsspec_low.spectrum, yerr=rmsspec_low.spectrum_error, label=rf"Rms $\Delta \nu=${lowfreq} Hz", color="purple")

ax.set_xscale('log')
ax.set_xlim(0.5, 10.)
ax.set_ylim(0.05, 0.28)

ax.set_ylabel("Fractional Covariance")
ax.set_xlabel("Energy (keV)")
ax.legend()
plt.show()
plt.close("all")

- Linear response in Fourier space

In [ ]:
## Compute transfer function H(nu) from an impulse response function h(t)

def transfer_function(irf, t):       # t is the array of times over which irf is defined
    dt = t[1] - t[0]                 # time bin of input irf
    H = np.fft.rfft(irf) * dt        # discrete fourier transform (factor dt adjusts normalization)
    nu = np.fft.rfftfreq(len(t), dt) # frequency array
    return nu, H

In [ ]:
## Interpolate H onto the desired frequency grid and compute lag

def lag_from_transfer_function(freq, nu_irf, H):
    
    H_interp = np.interp(freq, nu_irf, H.real) + 1j * np.interp(freq, nu_irf, H.imag)
    phase = np.angle(H_interp)
    lag = phase / (2 * np.pi * freq)
    return lag

In [ ]:
## Example 1: Top-hat 

dt = ..            # time resolution (s)
t_max = ..          # must be > tau_max
t = np.arange(0, t_max, dt)

tau_min = ..
tau_max = ..

irf_tophat = np.zeros_like(t)
mask = (t >= tau_min) & (t <= tau_max)
irf_tophat[mask] = 1.0 / (tau_max - tau_min)

nu_irf, H_tophat = transfer_function(irf_tophat, t)
lag_tophat = lag_from_transfer_function(cs_reb.freq, nu_irf, H_tophat)


In [ ]:
## Plot impulse response and lag-frequency spectrum

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)

ax2.plot(cs_reb.freq, lag_tophat, label='Top-hat IRF')
ax1.plot(t, irf_tophat, label='Top-hat IRF')

ax2.axhline(0, ls='--', color='k')
ax2.set_xscale('log')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Lag (s)')

ax1.set_xlabel('Time (s)')
ax1.set_ylabel('h(t)')

ax1.legend
ax2.legend()
plt.show()
plt.close("all")

In [ ]:
## Example 2: disc-like response

alpha = 1.5

irf_disc = np.zeros_like(t)
mask = (t >= tau_min) & (t <= tau_max)
irf_disc[mask] = t[mask] ** (-alpha)

# Normalize
irf_disc /= np.trapz(irf_disc, t)

nu_irf, H_disc = transfer_function(irf_disc, t)
lag_disc = lag_from_transfer_function(cs_reb.freq, nu_irf, H_disc)

In [ ]:
## Plot impulse response and lag-frequency spectrum

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), constrained_layout=True)

ax2.plot(cs_reb.freq, lag_disc, label='Disc IRF')
ax1.plot(t, irf_disc, label='Disc IRF')

ax2.axhline(0, ls='--', color='k')
ax2.set_xscale('log')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Lag (s)')

ax1.set_xlabel('Time (s)')
ax1.set_ylabel('h(t)')

ax2.legend()
plt.show()
plt.close("all")